In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from openbabel import openbabel
import pubchempy as pcp
import numpy as np
import py3Dmol
import os
import ipywidgets as widgets
from ipywidgets import interact, fixed, IntSlider, Text, Dropdown, ToggleButton, Button, FloatSlider, Checkbox
from IPython.display import display
obMol = openbabel.OBMol()
obConv = openbabel.OBConversion()
"""IMPORTANT: DO NOT USE ANY OTHER VARIABLES NAMED obMol OR obConv!!!"""

from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdCIPLabeler
from rdkit.Chem import rdAbbreviations
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.ipython_useSVG=True
IPythonConsole.molSize = 300,300


from pointgroup import PointGroup

In [7]:
class DataCache:
    def __init__(self):
        self.cache = {}

    def add_data(self, key, value):
        if key not in self.cache:
            self.cache[key] = [value]
        else:
            if value not in self.cache[key]:
                self.cache[key].append(value)

    def get_data(self, key):
        return self.cache.get(key)

    def print_data(self, target):
        if target == 'smiles':
            print(f"Data type: SMILES; Data: {self.cache['SMILES']}")
        elif target == 'cid':
            print(f"Data type: SMILES; Data: {self.cache['CID']}")
        elif target == 'inchi':
            print(f"Data type: SMILES; Data: {self.cache['InChi']}")
        elif target == 'inchikey':
            print(f"Data type: SMILES; Data: {self.cache['InChiKey']}")
        elif target == 'name':
            print(f"Data type: SMILES; Data: {self.cache['Name']}")
        elif target == 'all':
            for key, value in self.cache.items():
                print(f"Data type: {key}; Data: {value}")

In [8]:
class converter:
    def __init__(self, data: str, data_type: str, data_cache: DataCache):
        """Initializes the converter with the input data and its type"""
        self.data = data
        self.data_type = data_type.lower()
        self.data_cache = data_cache

    def convert(self, target_format: str):
        """Converts the input data to the target format"""
        target_format = target_format.lower()
        if self.data_type == 'name':
            self.data_cache.add_data('Name', self.data)
            smiles = self.name_to_smiles()
            sdf = self.name_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.data
        elif self.data_type == 'smiles':
            self.data_cache.add_data('SMILES', self.data)
            smiles = self.data
            sdf = self.smiles_to_sdf()
            if target_format == 'smiles':
                return self.data
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name1()
        elif self.data_type == 'inchi':
            self.data_cache.add_data('InChi', self.data)
            smiles = self.inchi_to_smiles()
            sdf = self.inchi_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.data
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)
        elif self.data_type == 'inchikey':
            self.data_cache.add_data('InChiKey', self.data)
            smiles = self.inchikey_to_smiles()
            sdf = self.inchikey_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.data
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)
        elif self.data_type == 'cid':
            self.data_cache.add_data('CID', self.data)
            smiles = self.cid_to_smiles()
            sdf = self.cid_to_sdf()
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.data
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)

    def name_to_smiles(self):
        """Converts molecule name to SMILES"""
        try:
            c = pcp.get_compounds(self.data, 'name')
            smiles = c[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def cid_to_smiles(self):
        """Converts CID to SMILES"""
        try:
            cid = pcp.get_compounds(self.data, 'cid')
            smiles = cid[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def smiles_to_name1(self):
        """Converts SMILES to molecule name"""
        try:
            smi = pcp.get_compounds(self.data, 'smiles')
            name = smi[0].iupac_name
            self.data_cache.add_data('Name', name)
            return name
        except IndexError:
            return None

    def smiles_to_name2(self, smiles):
        """Converts SMILES to molecule name"""
        try:
            smi = pcp.get_compounds(smiles, 'smiles')
            name = smi[0].iupac_name
            self.data_cache.add_data('Name', name)
            return name
        except IndexError:
            return None

    def inchi_to_smiles(self):
        """Converts InChi to SMILES"""
        try:
            ic = pcp.get_compounds(self.data, 'inchi')
            smiles = ic[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def inchikey_to_smiles(self):
        """Converts InChiKey to SMILES"""
        try:
            ick = pcp.get_compounds(self.data, 'inchikey')
            smiles = ick[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def smiles_to_cid(self, smiles):
        """Converts SMILES to CID"""
        try:
            c = pcp.get_cids(smiles, 'smiles', list_return='flat')
            cid = c[0]
            self.data_cache.add_data('CID', cid)
            return cid
        except IndexError:
            return None
        
    def smiles_to_sdf(self):
        """Converts SMILES to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'smiles', overwrite=True)
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def name_to_sdf(self):
        """Converts name to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'name', overwrite=True)
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def inchi_to_sdf(self):
        """Converts inchi to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'inchi', overwrite=True)
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def inchikey_to_sdf(self):
        """Converts inchikey to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'inchikey', overwrite=True)
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def cid_to_sdf(self):
        """Converts name to SDF"""
        try:
            file_path = f'./3Dfiles/{self.data}.sdf'
            try:
                pcp.download('SDF', file_path, self.data, 'cid', overwrite=True)
                with open(file_path, 'r') as f:
                    return f.read()
            except Exception as e:
                print("Error during download:", e)
                return None
        except IndexError:
            return None

    def smiles_to_inchi(self, smiles):
        """Converts SMILES to InChI"""
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("smiles", "inchi")
            obConv.ReadString(obMol, smiles)
            ic = obConv.WriteString(obMol)
            self.data_cache.add_data('InChi', ic)
            return ic
        except IndexError:
            return None

    def smiles_to_inchikey(self, smiles):
        """Converts SMILES to InChiKey"""
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("smiles", "inchikey")
            obConv.ReadString(obMol, smiles)
            ick = obConv.WriteString(obMol)
            self.data_cache.add_data('InChiKey', ick)
            return ick
        except IndexError:
            return None

    def sdf_to_xyz(self, sdf, smiles):
        """Converts SDF to XYZ"""
        name = self.smiles_to_name2(smiles)
        directory = './3Dfiles/'
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("sdf", "xyz")
            obConv.ReadString(obMol, sdf)
            xyz = obConv.WriteString(obMol)
            if not os.path.exists(directory):
                os.makedirs(directory)
            file_path = os.path.join('./3Dfiles/', f"{name}.xyz")
            with open(file_path, "w") as file:
                file.write(xyz)
            return xyz
        except IndexError:
            return None

    def sdf_to_zmat(self, sdf, smiles):
        """Converts SDF to zmat"""
        name = self.smiles_to_name2(smiles)
        directory = './3Dfiles/'
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("sdf", "gzmat")
            obConv.ReadString(obMol, sdf)
            zmat = obConv.WriteString(obMol)
            if not os.path.exists(directory):
                os.makedirs(directory)
            file_path = os.path.join('./3Dfiles/', f"{name}.txt")
            with open(file_path, "w") as file:
                file.write(zmat)
            return zmat
        except IndexError:
            return None

In [9]:
cache = DataCache()
sdf_cache = {}

In [10]:
def get_sdf(identifier, identifier_type):
    directory = './3Dfiles/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the file path for the SDF file
    sdf_file = os.path.join(directory, f"{identifier}.sdf")

    # Check if the SDF file already exists
    if not os.path.exists(sdf_file):
        # If not, convert SMILES to SDF and save it
        molecule = converter(identifier, identifier_type, cache)
        sdf = molecule.convert('sdf')
        if not sdf:
            print("Failed to convert molecule to SDF.")
            return None
        
        with open(sdf_file, 'w') as f:
            f.write(sdf)
    else:
        # If the SDF file exists, read it
        with open(sdf_file, 'r') as f:
            sdf = f.read()

    return sdf

In [11]:
style_dropdown = Dropdown(
    options=['line', 'stick', 'sphere','All'],
    value='All',
    description='Style:')

linewidth_slider = FloatSlider(
    value=2,
    min=1,
    max=10,
    step=1,
    description='Line Width')

radius_slider = FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    description='Atomic radius size') 

scale_slider = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    description='Sphere size')

NameError: name 'update_visibility' is not defined

NameError: name 'update_visibility' is not defined

NameError: name 'update_visibility' is not defined

NameError: name 'update_visibility' is not defined

In [10]:
def update_visibility(style):
    if style == 'line':
        linewidth_slider.layout.visibility = 'visible'
        radius_slider.layout.visibility = 'hidden'
        scale_slider.layout.visibility = 'hidden'
    elif style =='stick':
        linewidth_slider.layout.visibility = 'hidden'
        radius_slider.layout.visibility = 'visible'
        scale_slider.layout.visibility = 'hidden'
    elif style =='sphere':
        linewidth_slider.layout.visibility = 'hidden'
        radius_slider.layout.visibility = 'hidden'
        scale_slider.layout.visibility = 'visible'
    elif style == 'All':
        linewidth_slider.layout.visibility = 'visible'
        radius_slider.layout.visibility = 'visible'
        scale_slider.layout.visibility = 'visible'

In [12]:
def view3D(data_cache, identifier, identifier_type, style='All', linewidth='1', radius='0.2', scale='1'):
    """Visualize a molecule in 3D"""

    # Check if the SDF file is already cached
    if identifier in sdf_cache:
        sdf = sdf_cache[identifier]
    else:
        # Fetch the SDF file for the new molecule identifier
        sdf = get_sdf(identifier, identifier_type)
        # Cache the loaded SDF file
        sdf_cache[identifier] = sdf
    
    view = py3Dmol.view(width=1000, height=400)
    view.addModel(sdf,'sdf')
    view.setBackgroundColor('#000000', viewer=(0,0))
    
    if style == 'line':
        view.setStyle({'line': {'linewidth': linewidth}})
    elif style == 'stick':
        view.setStyle({'stick': {'radius': radius}})
    elif style == 'sphere':
        view.setStyle({'sphere': {'scale': scale}})
    elif style == 'All':
        view = py3Dmol.view(width=1500, height=400, viewergrid=(1,3), linked=True)
        view.addModel(sdf,'sdf')
        view.setBackgroundColor('#000000', viewer=(0,0))
        view.setBackgroundColor('#000000', viewer=(0,1))
        view.setBackgroundColor('#000000', viewer=(0,2))
        view.setStyle({'line': {'linewidth': linewidth}}, viewer=(0,0))
        view.setStyle({'stick':{'radius': radius}}, viewer=(0,1))
        view.setStyle({'sphere': {'scale': scale}}, viewer=(0,2))
    return view

interact(view3D, data_cache=fixed(cache), 
         identifier = input(),
         identifier_type=['Name', 'SMILES', 'InChi', 'InChiKey', 'CID'],
         style=style_dropdown,
         linewidth=linewidth_slider,
         radius=radius_slider,
         scale=scale_slider)

style_dropdown.observe(lambda change: update_visibility(change['new']), names='value')

interactive(children=(Text(value='benzene', description='identifier'), Dropdown(description='identifier_type',…

NameError: name 'update_visibility' is not defined

NameError: name 'update_visibility' is not defined

NameError: name 'update_visibility' is not defined

NameError: name 'update_visibility' is not defined

In [12]:
indexs_checkbox = Checkbox(
    value = False,
    description = 'Index atoms (small)'
)

indexl_checkbox = Checkbox(
    value = False,
    description = 'Index atoms (large)'
)

stereo_checkbox = Checkbox(
    value = False,
    description = 'Show stereochemistry' 
)

substructure_input = Text(
    value='',
    description='Enter SMILES substructure'
)

abbr_checkbox = Checkbox(
    value = False,
    description = 'Use abbreviations'
)

In [13]:
def indexs_atoms(molecule):
    IPythonConsole.drawOptions.addAtomIndices = True
    return molecule

def indexl_atoms(molecule):
    for atom in molecule.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return molecule

def stereo_atoms(molecule):
    IPythonConsole.drawOptions.addStereoAnnotation = True
    rdCIPLabeler.AssignCIPLabels(molecule)
    return molecule

def get_substructure(molecule, smarts):
    substructure = Chem.MolFromSmarts(smarts)
    return molecule.GetSubstructMatches(substructure)

def vis_abbr(molecule):
    abbrevs = rdAbbreviations.GetDefaultAbbreviations()
    return rdAbbreviations.CondenseMolAbbreviations(molecule, abbrevs)

In [14]:
def view2D(smi, indexs = False, indexl = False, stereo = False, abbreviations =  False, substructure=''):
    molecule = Chem.MolFromSmiles(smi)    
    if molecule:
        if indexs:
            molecule = indexs_atoms(molecule)
        elif indexs == False:
            IPythonConsole.drawOptions.addAtomIndices = False
        if indexl:
            molecule = indexl_atoms(molecule)
        if stereo:
            molecule = stereo_atoms(molecule)
        elif stereo == False:
            IPythonConsole.drawOptions.addStereoAnnotation = False
        if abbreviations:
            molecule = vis_abbr(molecule)
        if substructure:
            matches = get_substructure(molecule, substructure)
        return molecule
    else:
        print(f"The molecule name '{molecule_name}' is not valid.")
        return None

def handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure):
    return view2D(smi, indexs, indexl, stereo, abbreviations, substructure)

interact(handle_checkbox, smi='', 
         indexs=indexs_checkbox, 
         indexl=indexl_checkbox, 
         stereo=stereo_checkbox,
         abbreviations=abbr_checkbox,
         substructure=substructure_input)

interactive(children=(Text(value='', description='smi'), Checkbox(value=False, description='Index atoms (small…

<function __main__.handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure)>

In [38]:
from rdkit import Chem
from rdkit.Chem import Draw
from ipywidgets import interact, Checkbox, Text
from IPython.display import display

# Function to add indices to atoms
def index_atoms(molecule):
    for atom in molecule.GetAtoms():
        atom.SetProp('atomLabel', str(atom.GetIdx()))
    return molecule

# Function to add labels to atoms
def indexl_atoms(molecule):
    for atom in molecule.GetAtoms():
        atom.SetProp('atomLabel', atom.GetSymbol() + str(atom.GetIdx()))
    return molecule

# Function to handle stereochemistry annotations
def stereo_atoms(molecule):
    for bond in molecule.GetBonds():
        if bond.GetStereo() != Chem.rdchem.BondStereo.STEREONONE:
            bond.SetProp('bondNote', str(bond.GetStereo()))
    return molecule

# Function to visualize abbreviations
def vis_abbr(molecule):
    # This is a placeholder for the actual abbreviation visualization logic
    # You will need to implement the logic based on how you want to handle abbreviations
    return molecule

# Function to get substructure matches
def get_substructure(molecule, substructure):
    substructure_mol = Chem.MolFromSmarts(substructure)
    if substructure_mol is not None:
        return molecule.GetSubstructMatches(substructure_mol)
    else:
        print(f"The substructure '{substructure}' is not valid.")
        return []

# Example usage in the view2D function:
# if substructure:
#     matches = get_substructure(molecule, substructure)
#     molecule = Draw.rdMolDraw2D.PrepareAndDrawMolecule(molecule, highlightAtoms=matches)


# Improved view2D function
def view2D(smi, indexs=True, indexl=True, stereo=True, abbreviations=True, substructure=''):
    molecule = Chem.MolFromSmiles(smi)
    if molecule:
        Draw.PrepareMolForDrawing(molecule)  
        if indexs:
            molecule = indexs_atoms(molecule)
        if indexl:
            molecule = indexl_atoms(molecule)
        if stereo:
            molecule = stereo_atoms(molecule)
        if abbreviations:
            molecule = vis_abbr(molecule)
        if substructure:
            matches = get_substructure(molecule, substructure)
            molecule = Draw.rdMolDraw2D.PrepareAndDrawMolecule(molecule, highlightAtoms=matches)
        else:
            molecule = Draw.MolToImage(molecule)  
        display(molecule)  
    else:
        print(f"The SMILES string '{smi}' is not valid.")
        return None

# Function to handle the interaction with checkboxes
def handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure):
    view2D(smi, indexs, indexl, stereo, abbreviations, substructure)

# Widgets for user interaction
indexs_checkbox = Checkbox(description='Show Atom Indices')
indexl_checkbox = Checkbox(description='Show Label Indices')
stereo_checkbox = Checkbox(description='Show Stereochemistry')
abbr_checkbox = Checkbox(description='Show Abbreviations')
substructure_input = Text(description='Substructure')

# Interactive widget
interact(handle_checkbox, smi=Text(description='SMILES String'), 
         indexs=indexs_checkbox, 
         indexl=indexl_checkbox, 
         stereo=stereo_checkbox,
         abbreviations=abbr_checkbox,
         substructure=substructure_input)


interactive(children=(Text(value='', description='SMILES String'), Checkbox(value=False, description='Show Ato…

<function __main__.handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure)>

In [15]:
m2 = converter('C1CN2CC3=CCO[C@H]4CC(=O)N5[C@H]6[C@H]4[C@H]3C[C@H]2[C@@]61C7=CC=CC=C75', 'smiles', cache)

In [16]:
n = m2.convert('name')

In [17]:
m1 = converter('MEFKEPWMEQBLKI-AIRLBKTGSA-N', 'inchikey', cache)
m = m1.convert('smiles')
mn = m1.convert('name')
mi = m1.convert('inchi')

In [18]:
m = converter('Erythronolide B', 'name')
name = m.convert('name')
cid = m.convert('cid')
smi = m.convert('smiles')
print(smi)
#name_file = name + '.sdf'
#pcp.download('SDF', name_file, cid, 'cid', overwrite=True)

TypeError: converter.__init__() missing 1 required positional argument: 'data_cache'

In [ ]:
        if isinstance(self, (float,int)):
            raise ValueError(  
                f"Invalid input: '{self}'."
                " Input valid SMILES"
            )

        try:
            smi = str(self)
        
        except ValueError:     
            raise ValueError(  
                f"Invalid input: '{self}'."
                " `SMILES` must be a string, or convertible to a string."
                f" Original error message: {e}"
            )

        except IndexError:
            return None

In [52]:
"""SOME PROBLEMS TO FIX :
    - Benzene smiles C1=CC=CC=C1 gives D6h (correct)
    - But inputting "benzene" gives sometimes Cs sometimes Ci
    - Methane gives C1 instead of Td
    - Dones't work on SF6
"""

def get_molecule_name_from_smiles(smiles):
    compounds = pcp.get_compounds(smiles, 'smiles')
    if compounds:
        # Assuming the first compound is the one we want
        compound = compounds[0]
        return compound.iupac_name  # or compound.common_name for common name
    else:
        return "No compound found for the given SMILES."

def get_smiles_from_name_or_confirm_smiles(input_string):
    # This part checks if the input is already a valid SMILES.
    if Chem.MolFromSmiles(input_string) is not None:
        return input_string  # This will return the input SMILES.

    # If it's not, it will search to convert it to SMILES
    compounds = pcp.get_compounds(input_string, 'name')
    if compounds:
        compound = compounds[0]
        return compound.isomeric_smiles
    else:
        return None  # No valid compound was found for the input

def point_group_from_smiles(smiles):  

    # Convert SMILES to a molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print("Invalid SMILES input. Please provide a valid SMILES string.")
    else:
        # Add hydrogens to the molecule
        mol_with_h = Chem.AddHs(mol)

        # Generate 3D coordinates
        AllChem.EmbedMolecule(mol_with_h, AllChem.ETKDG())

        # Create a list to store coordinates
        coordinates_list = []
        atom_symbols = []
        
        # Iterate over atoms and get their coordinates
        for atom in mol_with_h.GetAtoms():
            pos = mol_with_h.GetConformer().GetAtomPosition(atom.GetIdx())
            coordinates_list.append([pos.x, pos.y, pos.z])
            atom_symbols.append(atom.GetSymbol())

    pg = PointGroup(positions= coordinates_list, symbols=atom_symbols)

    return pg.get_point_group()

if __name__ == "__main__":
    input_string = input("Input SMILES or molecule name: ")
    smiles_name = get_smiles_from_name_or_confirm_smiles(input_string)
    pg = point_group_from_smiles(smiles_name)
    mol_name = get_molecule_name_from_smiles(smiles_name)

    if pg:
        print(f"Point group of {mol_name} is {pg}")


[11:30:53] SMILES Parse Error: syntax error while parsing: Sulfurhexafluoride
[11:30:53] SMILES Parse Error: Failed parsing SMILES 'Sulfurhexafluoride' for input: 'Sulfurhexafluoride'


Point group of ethyl 2-oxohexanoate is C1
